In [2]:
# Read in Libraries
from __future__ import division, print_function
from logbook import Logger, StreamHandler
import sys
StreamHandler(sys.stdout).push_application()
log = Logger('Logbook')

import utils; reload(utils)
from utils import *
import gc
# from __future__ import division, print_function
from theano.sandbox import cuda
from vgg16bn import Vgg16BN
from sklearn import metrics

def accuracyfunc(y_act, y_pred):
    return metrics.accuracy_score(np.argmax(y_act, axis=1), np.argmax(y_pred, axis=1))


# Set Parameters and check files
input_exists = True
full = False
log.info('Set Paramters')
path = "../data/fish/"
batch_size=64

# Read in our VGG pretrained model
log.info('Get VGG')
model = vgg_ft_bn(8)

# Create our VGG model
log.info('Create VGG')
vgg640 = Vgg16BN((360, 640)).model
vgg640.pop()
vgg640.input_shape, vgg640.output_shape
vgg640.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])

# get labels
(val_classes, trn_classes, val_labels, trn_labels,
    val_filenames, filenames, test_filenames) = get_classes(path)

# Read in filenames
log.info('Read filenames')
raw_filenames = [f.split('/')[-1] for f in filenames]
raw_test_filenames = [f.split('/')[-1] for f in test_filenames]
raw_val_filenames = [f.split('/')[-1] for f in val_filenames]

[2017-02-09 23:22:06.691323] INFO: Logbook: Set Paramters
[2017-02-09 23:22:06.692352] INFO: Logbook: Get VGG


Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


[2017-02-09 23:22:09.926933] INFO: Logbook: Create VGG
Found 3086 images belonging to 8 classes.
Found 691 images belonging to 8 classes.
Found 1000 images belonging to 1 classes.
[2017-02-09 23:22:10.456098] INFO: Logbook: Read filenames


In [3]:
log.info('Read in data')
if not input_exists:

    batches = get_batches(path+'train', batch_size=batch_size)
    val_batches = get_batches(path+'valid', batch_size=batch_size*2, shuffle=False)
    (val_classes, trn_classes, val_labels, trn_labels, 
        val_filenames, filenames, test_filenames) = get_classes(path)
    
    # Fetch our large images 
    log.info('Fetch images')
    trn = get_data(path+'train', (360,640))
    val = get_data(path+'valid', (360,640))
    test = get_data(path+'test', (360,640))
    
    # Precompute the output of the convolutional part of VGG
    log.info('Get VGG output')
    conv_val_feat = vgg640.predict(val, batch_size=32, verbose=1)
    conv_trn_feat = vgg640.predict(trn, batch_size=32, verbose=1)
    conv_test_feat = vgg640.predict(test, batch_size=32, verbose=1)
    log.info('Write VGG output')
    save_array(path+'results/conv_val_feat.dat', conv_val_feat)
    save_array(path+'results/conv_trn_feat.dat', conv_trn_feat) 
    save_array(path+'results/conv_test_feat.dat', conv_test_feat)     

    # For memory purposes delete out the original train and validation
    log.info('Clear up memory')
    del trn, val, test
    gc.collect()

conv_val_feat = load_array(path+'results/conv_val_feat.dat')
conv_trn_feat = load_array(path+'results/conv_trn_feat.dat') 
conv_test_feat = load_array(path+'results/conv_test_feat.dat')

if full:
    conv_trn_feat = np.concatenate([conv_trn_feat, conv_val_feat])
    trn_labels = np.concatenate([trn_labels, val_labels]) 


[2017-02-07 21:49:00.531780] INFO: Logbook: Read in data


In [22]:
# Our Convolutional Net Architecture
log.info('Create and fit CNN')
def get_lrg_layers():
    return [
        BatchNormalization(axis=1, input_shape=conv_layers[-1].output_shape[1:]),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D(),
        Convolution2D(nf,3,3, activation='relu', border_mode='same'),
        BatchNormalization(axis=1),
        MaxPooling2D((1,2)),
        Convolution2D(8,3,3, border_mode='same'),
        Dropout(p),
        GlobalAveragePooling2D(),
        Activation('softmax')
    ]

# Set up the fully convolutional net (FCN); 
conv_layers,_ = split_at(vgg640, Convolution2D)
nf=128; p=0. # No dropout

lrg_model = []
predsls = []
pvalsls = []
bags = 3

for i in range(bags):
    log.info('Train round' + str(i))
    lrg_model.append(Sequential(get_lrg_layers()))
    #if i == 0:
	#lrg_model[i].summary()
    lrg_model[i].compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    lrg_model[i].fit(conv_trn_feat, trn_labels, batch_size=batch_size, nb_epoch=2, 
                 validation_data=(conv_val_feat, val_labels))
    lrg_model[i].optimizer.lr=1e-5
    lrg_model[i].fit(conv_trn_feat, trn_labels, batch_size=batch_size, nb_epoch=6,
                 validation_data=(conv_val_feat, val_labels))

    # Make our prediction on the lrg_model layer
    log.info('Output Prediction')
    predsls.append(lrg_model[i].predict(conv_test_feat, batch_size=batch_size)) # or try 32 batch_size
    pvalsls.append(lrg_model[i].predict(conv_val_feat, batch_size=batch_size))
    val_score = "%.3f" % metrics.log_loss(val_labels, sum(pvalsls)/len(pvalsls))
    acc_score = "%.3f" % accuracyfunc(val_labels, do_clip(sum(pvalsls)/len(pvalsls), .99))
    log.info('Bagged Validation Logloss ' + str(val_score))
    log.info('Bagged Validation Accuracy ' + str(acc_score))
    # 10 bagged : 0.131


[2017-02-07 22:23:12.302540] INFO: Logbook: Create and fit CNN
[2017-02-07 22:23:12.304608] INFO: Logbook: Train round0
th
Train on 3277 samples, validate on 500 samples
Epoch 1/2
3277/3277 [==============================] - 16s - loss: 0.6367 - acc: 0.7952 - val_loss: 1.6308 - val_acc: 0.5600
Epoch 2/2
3277/3277 [==============================] - 16s - loss: 0.1219 - acc: 0.9710 - val_loss: 0.3252 - val_acc: 0.9200
Train on 3277 samples, validate on 500 samples
Epoch 1/6
3277/3277 [==============================] - 16s - loss: 0.0302 - acc: 0.9924 - val_loss: 0.2971 - val_acc: 0.9240
Epoch 2/6
3277/3277 [==============================] - 16s - loss: 0.0258 - acc: 0.9936 - val_loss: 0.1600 - val_acc: 0.9620
Epoch 3/6
3277/3277 [==============================] - 16s - loss: 0.0048 - acc: 0.9994 - val_loss: 0.1730 - val_acc: 0.9640
Epoch 4/6
3277/3277 [==============================] - 16s - loss: 0.0016 - acc: 1.0000 - val_loss: 0.1488 - val_acc: 0.9700
Epoch 5/6
3277/3277 [============

In [25]:
conv_pseu_feat = np.concatenate([conv_trn_feat, conv_test_feat])
pseu_labels = np.concatenate([trn_labels, sum(predsls)/len(predsls)])
pseu_labels[:]

array([[  1.0000e+00,   0.0000e+00,   0.0000e+00, ...,   0.0000e+00,   0.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00,   0.0000e+00, ...,   0.0000e+00,   0.0000e+00,   0.0000e+00],
       [  1.0000e+00,   0.0000e+00,   0.0000e+00, ...,   0.0000e+00,   0.0000e+00,   0.0000e+00],
       ..., 
       [  1.8985e-01,   5.3936e-02,   4.4636e-03, ...,   5.1575e-01,   4.8795e-02,   2.4992e-02],
       [  9.9704e-01,   1.6194e-04,   3.0691e-04, ...,   1.5662e-03,   3.8003e-05,   4.8699e-04],
       [  8.3429e-03,   1.2262e-01,   4.4068e-03, ...,   7.5299e-02,   1.9163e-02,   5.6677e-01]])

In [26]:
lrg_model = []
predsls = []
pvalsls = []
bags = 3

for i in range(bags):
    log.info('Train round' + str(i))
    lrg_model.append(Sequential(get_lrg_layers()))
    #if i == 0:
	#lrg_model[i].summary()
    lrg_model[i].compile(Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    lrg_model[i].fit(conv_pseu_feat, pseu_labels, batch_size=batch_size, nb_epoch=2, 
                 validation_data=(conv_val_feat, val_labels))
    lrg_model[i].optimizer.lr=1e-5
    lrg_model[i].fit(conv_pseu_feat, pseu_labels, batch_size=batch_size, nb_epoch=6,
                 validation_data=(conv_val_feat, val_labels))

    # Make our prediction on the lrg_model layer
    log.info('Output Prediction')
    predsls.append(lrg_model[i].predict(conv_test_feat, batch_size=batch_size)) # or try 32 batch_size
    pvalsls.append(lrg_model[i].predict(conv_val_feat, batch_size=batch_size))
    val_score = "%.3f" % metrics.log_loss(val_labels, sum(pvalsls)/len(pvalsls))
    acc_score = "%.3f" % accuracyfunc(val_labels, do_clip(sum(pvalsls)/len(pvalsls), .99))
    log.info('Bagged Validation Logloss ' + str(val_score))
    log.info('Bagged Validation Accuracy ' + str(acc_score))
    # 10 bagged : 0.131

[2017-02-07 22:31:30.784769] INFO: Logbook: Train round0
th
Train on 4277 samples, validate on 500 samples
Epoch 1/2
4277/4277 [==============================] - 20s - loss: 0.6741 - acc: 0.7891 - val_loss: 1.3515 - val_acc: 0.6960
Epoch 2/2
4277/4277 [==============================] - 20s - loss: 0.2161 - acc: 0.9551 - val_loss: 0.4706 - val_acc: 0.8340
Train on 4277 samples, validate on 500 samples
Epoch 1/6
4277/4277 [==============================] - 20s - loss: 0.1682 - acc: 0.9687 - val_loss: 0.1947 - val_acc: 0.9580
Epoch 2/6
4277/4277 [==============================] - 20s - loss: 0.1516 - acc: 0.9769 - val_loss: 0.2063 - val_acc: 0.9580
Epoch 3/6
4277/4277 [==============================] - 20s - loss: 0.1450 - acc: 0.9759 - val_loss: 0.1930 - val_acc: 0.9540
Epoch 4/6
4277/4277 [==============================] - 20s - loss: 0.1406 - acc: 0.9785 - val_loss: 0.1650 - val_acc: 0.9660
Epoch 5/6
4277/4277 [==============================] - 20s - loss: 0.1311 - acc: 0.9801 - val_lo

In [28]:
# metrics.log_loss(val_labels, do_clip(sum(pvalsls)/len(pvalsls), .9999))
preds = sum(predsls)/len(predsls)
subm = do_clip(preds,0.999)
subm_name = path+'results/subm_bb_conv_lrg0207pseu1.csv.gz'
pred_name = path+'results/pred_bb_conv_lrg0207pseu1.csv.gz'

classes = ['ALB', 'BET', 'DOL', 'LAG', 'NoF', 'OTHER', 'SHARK', 'YFT']
submission = pd.DataFrame(subm, columns=classes)
submission.insert(0, 'image', raw_test_filenames)
submission.to_csv(subm_name, index=False, compression='gzip')
subm1 = pd.DataFrame(preds, columns=classes)
subm1.insert(0, 'image', raw_test_filenames)
subm1.to_csv(pred_name, index=False, compression='gzip')

log.info('Done - files @ ' + subm_name)

# Bag 6 Original scores 
#[2017-02-07 16:06:28.804177] INFO: Logbook: Bagged Validation Logloss 0.169
#[2017-02-07 16:06:28.804338] INFO: Logbook: Bagged Validation Logloss 0.978

[2017-02-07 22:41:16.203912] INFO: Logbook: Done - files @ ../data/fish/results/subm_bb_conv_lrg0207pseu1.csv.gz


In [4]:
subm_name = '../data/fish/results/subm_bb_conv_lrg0206A.csv.gz'
FileLink(subm_name)

/home/ubuntu/fish/data/fish/results/subm_bb_conv_lrg0206A.csv.gz